In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


import matplotlib
matplotlib.use('Agg') 

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

from math import sqrt
import tensorflow as tf
from scipy.stats import skew
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_excel("dataset_irradiance_v2.xlsx", index_col = [0])
df

,GHI,DNI,DHI,Temperatura en 2.0 metros [mean],Humedad Relativa en 2.0 metros [mean],Velocidad de viento en 6.0 metros [mean],Velocidad de viento en 12.0 metros [mean],Dirección del viento en 12.0 metros [mean],Voltaje Bateria en 0.0 metros [mean],Voltaje Bateria en 0.0 metros [mean].1
Fecha,,,,,,,,,,
2021-01-14 15:00:00,1071.0,1012.0,160.6,33.30,14.920,3.528,3.601,350.600,12.42,36.59
2021-01-14 15:10:00,1027.0,1003.0,164.8,33.24,14.960,3.925,4.161,5.782,12.42,36.48
2021-01-14 15:20:00,1014.0,974.0,179.2,32.89,15.180,4.142,4.491,27.450,12.41,36.20
2021-01-14 15:30:00,880.0,822.0,194.3,-11.90,-0.454,5.092,5.320,20.760,12.41,35.73
2021-01-14 15:40:00,742.4,669.7,199.7,-265.20,-119.600,5.357,5.679,33.330,12.42,35.58
...,...,...,...,...,...,...,...,...,...,...
2022-01-03 09:10:00,716.8,953.0,79.7,21.07,25.100,0.963,0.795,247.700,12.40,37.25
2022-01-03 09:20:00,755.7,961.0,80.4,21.72,23.930,0.865,0.786,226.200,12.39,39.39
2022-01-03 09:30:00,795.0,971.0,81.7,22.60,22.740,0.754,0.591,314.400,12.39,41.30


In [3]:
df.isna().sum()

GHI                                           0
DNI                                           0
DHI                                           0
Temperatura en 2.0 metros [mean]              0
Humedad Relativa en 2.0 metros [mean]         1
Velocidad de viento en 6.0 metros [mean]      0
Velocidad de viento en 12.0 metros [mean]     0
Dirección del viento en 12.0 metros [mean]    0
Voltaje Bateria en 0.0 metros [mean]          0
Voltaje Bateria en 0.0 metros [mean].1        0
dtype: int64

In [4]:
df = df.dropna()
df = df.drop('Voltaje Bateria en 0.0 metros [mean]', axis=1)
df

,GHI,DNI,DHI,Temperatura en 2.0 metros [mean],Humedad Relativa en 2.0 metros [mean],Velocidad de viento en 6.0 metros [mean],Velocidad de viento en 12.0 metros [mean],Dirección del viento en 12.0 metros [mean],Voltaje Bateria en 0.0 metros [mean].1
Fecha,,,,,,,,,
2021-01-14 15:00:00,1071.0,1012.0,160.6,33.30,14.920,3.528,3.601,350.600,36.59
2021-01-14 15:10:00,1027.0,1003.0,164.8,33.24,14.960,3.925,4.161,5.782,36.48
2021-01-14 15:20:00,1014.0,974.0,179.2,32.89,15.180,4.142,4.491,27.450,36.20
2021-01-14 15:30:00,880.0,822.0,194.3,-11.90,-0.454,5.092,5.320,20.760,35.73
2021-01-14 15:40:00,742.4,669.7,199.7,-265.20,-119.600,5.357,5.679,33.330,35.58
...,...,...,...,...,...,...,...,...,...
2022-01-03 09:10:00,716.8,953.0,79.7,21.07,25.100,0.963,0.795,247.700,37.25
2022-01-03 09:20:00,755.7,961.0,80.4,21.72,23.930,0.865,0.786,226.200,39.39
2022-01-03 09:30:00,795.0,971.0,81.7,22.60,22.740,0.754,0.591,314.400,41.30


### Definición de target y predictores

### 60 minutos

In [5]:
df_target_ghi = df["GHI"]
df_target_ghi = df_target_ghi.shift(-6)
df_target_dni = df["DNI"]
df_target_dni = df_target_dni.shift(-6)

df_predictores = df

In [6]:
print('GHI max: ', df_target_ghi.max())
print('DNI max: ', df_target_dni.max())

GHI max:  1288.0
DNI max:  1135.0


In [7]:
df_target_ghi.drop(df_target_ghi.index[-6], inplace = True)
df_target_dni.drop(df_target_dni.index[-6], inplace = True)
df_predictores.drop(df_predictores.index[-6], inplace = True)

### Normalizar

In [8]:
df_target_ghi = df_target_ghi/1288.0
df_target_dni = df_target_dni/1135

### Separación períodos de tiempo a usar

In [9]:
df_ml_predictores = df_predictores.loc["2021-01-14":"2021-11-30"]
df_ml_target_ghi = df_target_ghi.loc["2021-01-14":"2021-11-30"]
df_ml_target_dni = df_target_dni.loc["2021-01-14":"2021-11-30"]

In [10]:
df_post_predictores = df_predictores.loc["2021-12-01":"2021-12-31"]
df_post_target_ghi = df_target_ghi.loc["2021-12-01":"2021-12-31"]
df_post_target_dni = df_target_dni.loc["2021-12-01":"2021-12-31"]

## Separación set de entrenamiento y testeo

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_ml_predictores, df_ml_target_dni, test_size = 0.2, random_state = 40)

## Selección de hiperparámetros (valor de k)

In [12]:
# error = []
# for k in range(1, 40):
#     kNN = KNeighborsRegressor(n_neighbors = k, weights='distance')
#     y_predict = cross_val_predict(kNN, X_train, y_train, cv = 10)
#     error.append(mean_squared_error(y_train, y_predict))

In [13]:
# plt.plot(range(1, 40), error)
# plt.savefig('errorkNN.png')

### Valor con menor error, k = 7

## Modelo k- Nearest Neighboors

In [14]:
model_kNN = KNeighborsRegressor(n_neighbors = 7, weights='distance')

## Entrenamiento

In [15]:
kNN_fit = model_kNN.fit(X_train, y_train)

## Evaluación

In [16]:
kNN_evaluate = model_kNN.predict(X_test)

## Predicción

In [17]:
#prediccion para grafico de dispersion
pred_kNN = model_kNN.predict(df_post_predictores)
df_post_predictores_g1 = df_post_predictores.loc["2021-12-22":"2021-12-24"]
df_post_target_ghi_g1 = df_post_target_ghi.loc["2021-12-22":"2021-12-24"]
df_post_target_dni_g1 = df_post_target_dni.loc["2021-12-22":"2021-12-24"]
#prediccion para grafico ejemplo de dias nublados
pred_kNN_g1 = model_kNN.predict(df_post_predictores_g1)

## Comparación

In [18]:
font = {'family': 'serif',
        'color':  'blue',
        'weight': 'normal',
        'size': 16,
        'alpha': 0.3
        }

fig = plt.figure(figsize = (10*(1+np.sqrt(5))/2,10))



plt.plot(df_post_target_dni_g1, '-k', label = r'Medición')
plt.plot(df_post_target_dni_g1.index[1::],pred_kNN_g1[0:-1], '-.', label = r'kNN')

plt.legend(fontsize = 20)



#plt.title(r'Comparación con datos reales del 2021-12-22 - 2021-12-24')
plt.xlabel(r'Fechas', fontsize = 23)
plt.ylabel(r'Irradiancia normal directa DNI normalizada', fontsize = 23)

# plt.show()

plt.savefig('Comparacion_datos_kNN_60min_DNI.png')

In [19]:
def MBE(y_true, y_pred):
 
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true = y_true.reshape(len(y_true),1)
    y_pred = y_pred.reshape(len(y_pred),1)   
    diff = (y_pred-y_true)
    mbe = diff.mean()
    print('MBE = ', mbe)
    
def skewness(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true = y_true.reshape(len(y_true),1)
    y_pred = y_pred.reshape(len(y_pred),1)   
    diff = (y_pred-y_true)
    skewness = skew(diff) 
    print('Skewness = ', skewness)

In [20]:
print('La correlación de la serie de datos con la predicción es: ', np.round(np.corrcoef(np.squeeze(df_post_target_dni[1::].values), np.squeeze(pred_kNN[0:-1]))[0,1],3))
print('MAE test: ', mean_absolute_error(y_test, kNN_evaluate))
print('RMSE test: ', sqrt(mean_squared_error(y_test, kNN_evaluate)))
MBE(df_post_target_dni, pred_kNN)
skewness(df_post_target_dni, pred_kNN)


La correlación de la serie de datos con la predicción es:  0.933
MAE test:  0.042594987634004766
RMSE test:  0.10010265851742584
MBE =  0.013619632586983487
Skewness =  [0.95571911]


In [21]:
font = {'family': 'serif',
        'color':  'blue',
        'weight': 'normal',
        'size': 16,
        'alpha': 0.3
        }

fig = plt.figure(figsize = (10*(1+np.sqrt(5))/2,10))


plt.scatter(df_post_target_dni, pred_kNN)
plt.plot(df_post_target_ghi, df_post_target_ghi, 'k')


plt.xlim(0, 1)
plt.ylim(0, 1)

#plt.title(r'Comparación con datos reales del 2021-12-22 - 2021-12-24')
plt.xlabel(r'Irradiancia DNI real normalizada', fontsize = 23)
plt.ylabel(r'Irradiancia DNI predicha normalizada', fontsize = 23)


plt.savefig('dispersion_knn_dni_60.png')